In [ ]:
import requests
from bs4 import BeautifulSoup
from flask import Flask, request

def 取得股票股利資料(stock_code):

    # 根據股票的代號建立 URL
    url = f"https://tw.stock.yahoo.com/quote/{stock_code}/dividend"

    # 發送 GET 請求獲取網頁內容
    response = requests.get(url)

    # Http Status Code 200 OK
    if response.status_code == 200:
        # 解析 HTML 內容
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 提取股價，根據 Yahoo 股市網頁結構，上漲、下跌、平盤股價在不同的 class 中
        stock_price_up = soup.find('span', {'class' : 'Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-up)'})
        stock_price_down = soup.find('span', {'class' : 'Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-down)'})
        stock_price_flat = soup.find('span', {'class' : 'Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c)'})
        
        # 根據不同的狀態來抓取股價
        if stock_price_up:
            stock_price = float(stock_price_up.text.strip().replace(',', ''))  # 上漲
        elif stock_price_down:
            stock_price = float(stock_price_down.text.strip().replace(',', ''))  # 下跌
        elif stock_price_flat:
            stock_price = float(stock_price_flat.text.strip().replace(',', ''))  # 平盤
        else:
            print(f"{stock_code}未找到股價")
            return None
        
        # 找到包含股利資料的 <p> 標籤
        dividend_section = soup.find('p', {'class' : 'Mb(20px) Mb(12px)--mobile Fz(16px) Fz(18px)--mobile C($c-primary-text)'})
        
        if dividend_section:
            # 提取股利資料
            # 找到所有 <span class="Fw(b)"> 標籤，這些標籤包含我們需要的數據
            data_spans = dividend_section.find_all('span', {'class' : 'Fw(b)'})
            
            # 檢查是否找到足夠的數據
            if len(data_spans) >= 4:
                # 連續發放股利年數
                years_of_dividend = data_spans[0].text.strip()
                # 合計發放股利金額
                total_dividend = data_spans[1].text.strip()
                # 近 5 年平均現金殖利率
                average_dividend_yield = float(data_spans[3].text.strip().replace('%', '')) / 100  # 轉換為小數
                
                # 顯示抓取到的資料
                # print(f"股票代號: {stock_code}", f"股價: {stock_price}")
                # print(f"連續發放股利年數: {years_of_dividend}")
                # print(f"合計發放股利金額: {total_dividend} 元")
                # print(f"近 5 年平均現金殖利率: {average_dividend_yield}")

                return {
                    "股票代號": stock_code,
                    "股價": stock_price,
                    "連續發放股利年數": years_of_dividend,
                    "合計發放股利金額": total_dividend,
                    "近5年平均現金殖利率": average_dividend_yield
                }
            else:
                print(f"{stock_code}未能找到完整的股利資料")
                return None
        else:
            print(f"{stock_code}未找到股利資料區域")
            return None
    else:
        print(f"{stock_code}無法獲取網頁內容，請檢查網站連接")
        return None

# 計算便宜價、合理價、昂貴價
def calculate_prices(dividend_per_share, target_yield, market_average_yield, low_target_yield):
    
    cheap_price = dividend_per_share / target_yield
    fair_price = dividend_per_share / market_average_yield
    expensive_price = dividend_per_share / low_target_yield
    
    return cheap_price, fair_price, expensive_price

# 個人設定的目標殖利率
# 根據證交所統計, 台股整體殖利率自 2014 年至 2023 年 7 月近十年的平均為 3.94%, 而自 2018 年至 2023 年 7 月近五年的台股平均殖利率約 3.83%
# 用台灣銀行數位存款利率當最低標準
def 計算股價(stock_code, target_yield = 0.06, market_average_yield = 0.038, low_target_yield = 0.03):

    stock_data = 取得股票股利資料(stock_code)

    if stock_data:
        # 計算平均股利
        average_dividend = stock_data["股價"] * stock_data["近5年平均現金殖利率"]
        # print(f"股票代號 {stock_code} 平均股利: {average_dividend:.2f} 元")
        
        # 計算便宜價、合理價、昂貴價
        cheap, fair, expensive = calculate_prices(average_dividend, target_yield, market_average_yield, low_target_yield)
        # 顯示價格
        # print(f"股票代號 {stock_code} 的便宜價 : {cheap :.2f} 元")
        # print(f"股票代號 {stock_code} 的合理價 : {fair :.2f} 元")
        # print(f"股票代號 {stock_code} 的昂貴價 : {expensive :.2f} 元")
        return f"股票代號 {stock_code} 的便宜價 : {cheap :.2f} 元, 合理價 : {fair :.2f} 元, 昂貴價 : {expensive :.2f} 元"
    
    else:
        msg = f"{stock_code}無法獲取股利資料，請檢查股票代號或網站連接"
        print(msg)
        return msg

# print(計算股價("2881"))  # 台積電

# 初始化 Flask 應用程式
app = Flask(__name__)

# 設定 LINE BOT Token
BOT_TOKEN = "1JJuR+VMnP4gftJE1z3yLHHsI4HEm3j2kDeyBSOkV+D3Nyok2q9CSVp4HHDpkpw0FOsgzP96nQBx63mIHh+3B/SBUGsjNZ3LuhXcaSaYDzGDUK0lwiIeTlqtJBkWlerAyhORneeNgw2wKBcJ1WW4UAdB04t89/1O/w1cDnyilFU="
ngrok_port = 9527

# 傳送文字訊息函數
def send_text_message(reply_token, text):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + BOT_TOKEN
    }
    
    payload = {
        "replyToken": reply_token,
        "messages": [{
            "type": "text",
            "text": text
        }
     ]
    }
    
    # 發送 POST 請求至 LINE Messaging API
    response = requests.post(
        "https://api.line.me/v2/bot/message/reply",
        headers=headers,
        json=payload
    )
    return response

# LINE Webhook 入口
@app.route("/", methods = ['POST'])
def linebot():
    # 取得使用者傳來的資料
    data = request.get_json()
    print(data)
    
    # 提取 replyToken
    reply_token = data['events'][0]['replyToken']
    
    # 回傳文字訊息
    response = send_text_message(reply_token, 計算股價(data['events'][0]['message']['text']))
    
    if response.status_code == 200:
        return "OK", 200
    else:
        print("發送訊息失敗:", response.status_code, response.text)
        return "Error", 400

# 啟動 Flask 伺服器
if __name__ == "__main__":
    app.run(port = ngrok_port)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:9527
Press CTRL+C to quit


{'destination': 'Uc6b82425fd2239340a09c0db9a9af7c5', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '572694222197752035', 'quoteToken': 'Bgbu0-al-RHnwmpdrAd0vvcRXKKlA0mx52yS-2ItznHUYbcAR68j3Q1Wc7N7TSd3R4o5DSDonlIaVPw7LrgpS5gb6G2oqBCM5A_Dnris4gO2rVWF3FrPr2DBCRo64o4v13YazUMKsbq4gWx4CBrbWQ', 'text': '2330'}, 'webhookEventId': '01K1PQWXM210FNK6NJ371EJ5QE', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1754183660923, 'source': {'type': 'user', 'userId': 'U7aaa9d944ea3440663f01be0af6f2304'}, 'replyToken': '0e32df56ed5841fead4ccb84e6e8f6a6', 'mode': 'active'}]}


127.0.0.1 - - [03/Aug/2025 09:14:23] "POST / HTTP/1.1" 200 -


{'destination': 'Uc6b82425fd2239340a09c0db9a9af7c5', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '572694478805794902', 'quoteToken': 'YyUPHiG209Ws1tltC5949tXRsLy3z7gZDGpsfzbdQ-tKHUOWTCs9amEsM9YRuK2iNjNaqWWvFXiOu7l_3Y_56nL7tG9RKIr6WrMIi9C53ahehnJUGoWicB5M_e0DUrwzyBvuB6h_lQP2bJC-yj0UuA', 'text': '2881'}, 'webhookEventId': '01K1PR1K3GQ22W80SNH3QPGT25', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1754183813748, 'source': {'type': 'user', 'userId': 'U7aaa9d944ea3440663f01be0af6f2304'}, 'replyToken': 'a28ea79242ac469fab701db332bcb802', 'mode': 'active'}]}


127.0.0.1 - - [03/Aug/2025 09:16:56] "POST / HTTP/1.1" 200 -


{'destination': 'Uc6b82425fd2239340a09c0db9a9af7c5', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '572695226717306983', 'quoteToken': '5qVognioheWDcDT64NJw07RUqkRR52N66-l8ePD4UTIMScaUwLuvrNr6o9Zvx-fpPslMBtJYQT_1Cfo1iLteTA6EsrhdCz49vjISCnwMPX67JQo-_JSdwhiqpmS_wc2uD2KNX1Z8tyX8F70HeefvAQ', 'text': '2888'}, 'webhookEventId': '01K1PRF61DQ62EXJS1TCF6KWGG', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1754184259540, 'source': {'type': 'user', 'userId': 'U7aaa9d944ea3440663f01be0af6f2304'}, 'replyToken': '2f8c52600f9a4f52b012592997c6780b', 'mode': 'active'}]}
2888無法獲取網頁內容，請檢查網站連接
2888無法獲取股利資料，請檢查股票代號或網站連接


127.0.0.1 - - [03/Aug/2025 09:24:21] "POST / HTTP/1.1" 400 -


發送訊息失敗: 400 {"message":"The request body has 1 error(s)","details":[{"message":"May not be empty","property":"messages[0].text"}]}
